In [2]:
from transition1x import Dataloader

In [ ]:
from torch_geometric.data import Data
import torch

In [ ]:

import numpy as np

dataloader = Dataloader("data/transition1x.h5", datasplit="test", only_final=True)

from sklearn.preprocessing import OneHotEncoder
from ase import Atoms
import ase.io
atom_encoder = OneHotEncoder(sparse_output=False)
atom_encoder.fit(np.array([1, 6, 7, 8]).reshape(-1,1))
idx = 0
dataset = []
for molecule in dataloader:
    z_r = atom_encoder.transform(np.array(molecule["reactant"]["atomic_numbers"]).reshape(-1, 1))
    padded_z_r = np.zeros((len(molecule["reactant"]["atomic_numbers"]), 5))
    padded_z_r[:,:z_r.shape[1]] = z_r

    z_p = atom_encoder.transform(np.array(molecule["product"]["atomic_numbers"]).reshape(-1, 1))
    padded_z_p = np.zeros((len(molecule["product"]["atomic_numbers"]), 5))
    padded_z_p[:,:z_p.shape[1]] = z_p

    z_t = atom_encoder.transform(np.array(molecule["transition_state"]["atomic_numbers"]).reshape(-1, 1))
    padded_z_t = np.zeros((len(molecule["transition_state"]["atomic_numbers"]), 5))
    padded_z_t[:,:z_t.shape[1]] = z_t
    data = Data(
        rxn = molecule['rxn'],
        E_transition_state = torch.tensor(molecule["transition_state"]["wB97x_6-31G(d).atomization_energy"], dtype=torch.float32),
        E_reactant = torch.tensor(molecule["reactant"]["wB97x_6-31G(d).atomization_energy"], dtype=torch.float32),
        E_product = torch.tensor(molecule["product"]["wB97x_6-31G(d).atomization_energy"], dtype=torch.float32),

        pos_transition_state = torch.tensor(molecule["transition_state"]["positions"], dtype=torch.float32),
        formula_transition_state = molecule["transition_state"]["formula"],
        z_transition_state = torch.tensor(padded_z_t, dtype=torch.float32),

        pos_reactant = torch.tensor(molecule["reactant"]["positions"], dtype=torch.float32),
        formula_reactant = molecule["reactant"]["formula"],
        z_reactant = torch.tensor(padded_z_r, dtype=torch.float32),
        
        pos_product = torch.tensor(molecule["product"]["positions"], dtype=torch.float32),
        formula_product = molecule["product"]["formula"],
        z_product = torch.tensor(padded_z_p, dtype=torch.float32),
        
    )
    atoms = Atoms(molecule["reactant"]["formula"], positions=molecule["reactant"]["positions"])
    ase.io.write(f"{molecule['rxn']}-{idx}.xyz", atoms, format="xyz", append=True)
    atoms = Atoms(molecule["transition_state"]["formula"], positions=molecule["transition_state"]["positions"])
    ase.io.write(f"{molecule['rxn']}-{idx}.xyz", atoms, format="xyz", append=True)
    atoms = Atoms(molecule["product"]["formula"], positions=molecule["product"]["positions"])
    ase.io.write(f"{molecule['rxn']}-{idx}.xyz", atoms, format="xyz", append=True)
    idx += 1
    dataset.append(data)

In [ ]:
torch.save(dataset, "data/test.pt")

In [ ]:

from mdgen.dataset import EquivariantTransformerDataset_Transition1x
data_dir = "data"
dataset = EquivariantTransformerDataset_Transition1x(data_dirname=data_dir, sim_condition=False, tps_condition=True, num_species=5, stage="test")

tps_masked_dataset = []
for i in range(len(dataset)):
    tps_masked_dataset.append(dataset[i])

torch.save(tps_masked_dataset, f"{data_dir}/tps_masked_test.pt")

In [ ]:
E_reactant = [data.E_reactant for data in dataset]
E_product = [data.E_product for data in dataset]
E_transition_state = [data.E_transition_state for data in dataset]

In [ ]:
import matplotlib.pyplot as plt
_ = plt.hist(E_reactant)
_ = plt.hist(E_product)
_ = plt.hist(E_transition_state)

In [ ]:

_ = plt.hist(np.array(E_transition_state) - np.array(E_reactant))